In [1]:
# 문장단위 분류가 아닌, 토큰 단위의 분류를 진행하는 태스크
# 하나의 문장을 입력으로 받고, 각각의 토큰을 각각 분류
# 주로 문장 내에서 유효한 개체를 추출해 내는 개체명 인식 태스크에 많이 사용

In [5]:
# 문장분류 모댈과 비슷하지만, 벡터 차원 축소(풀링) 작업을 진행하지 않고 모든 토큰에 각각 출력 헤더를 달아 독립적으로 분류 진행
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "klue/bert-base"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

model # 마지막에 classifier 추가되어 있음

2025-06-16 05:31:56.749214: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-16 05:31:56.756085: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750051916.764462   37027 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750051916.766922   37027 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750051916.773402   37027 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [8]:
from datasets import load_dataset

dataset = load_dataset("klue", "ner")

sample = dataset['train'][0]
print('tokens : ', sample['tokens'][:20])
print('ner tags: ', sample['ner_tags'][:20])
print((len(sample['tokens']), (len(sample['tokens']))))

tokens :  ['특', '히', ' ', '영', '동', '고', '속', '도', '로', ' ', '강', '릉', ' ', '방', '향', ' ', '문', '막', '휴', '게']
ner tags:  [12, 12, 12, 2, 3, 3, 3, 3, 3, 12, 2, 3, 12, 12, 12, 12, 2, 3, 3, 3]
(66, 66)


In [9]:
for i in range(len(sample['ner_tags'])):
    print(sample['tokens'][i], '\t', sample['ner_tags'][i])

특 	 12
히 	 12
  	 12
영 	 2
동 	 3
고 	 3
속 	 3
도 	 3
로 	 3
  	 12
강 	 2
릉 	 3
  	 12
방 	 12
향 	 12
  	 12
문 	 2
막 	 3
휴 	 3
게 	 3
소 	 3
에 	 12
서 	 12
  	 12
만 	 2
종 	 3
분 	 3
기 	 3
점 	 3
까 	 12
지 	 12
  	 12
5 	 8
㎞ 	 9
  	 12
구 	 12
간 	 12
에 	 12
는 	 12
  	 12
승 	 12
용 	 12
차 	 12
  	 12
전 	 12
용 	 12
  	 12
임 	 12
시 	 12
  	 12
갓 	 12
길 	 12
차 	 12
로 	 12
제 	 12
를 	 12
  	 12
운 	 12
영 	 12
하 	 12
기 	 12
로 	 12
  	 12
했 	 12
다 	 12
. 	 12


In [12]:
# 이미 문자 단위로 분할되었으므로 tokenize 되어있다고 봐야 함. 따라서 토큰화 건너뛰고 바로 정수 인코딩으로 넘어감
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    labels=[]
    for i,label in enumerate(examples[f"ner_tags"]):
        # 토큰을 해당 단어에 매핑
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        # 특수토큰을 -100으로 세팅
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(12)
            #주어진 단어의 첫번째 토큰에만 레이블 지정
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset['train'].column_names
)

Map:   0%|          | 0/21008 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [13]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
batch = data_collator([tokenized_dataset['train'][i] for i in range(10)])

In [14]:
# klue/bert 모델은 id2label이 세팅되어 있지 않으므로 지정해 주어야 함
id2label = {
    0: "B-DT",
    1: "I-DT",
    2: "B-LC",
    3: "I-LC",
    4: "B-OG",
    5: "I-OG",
    6: "B-PS",
    7: "I-PS",
    8: "B-QT",
    9: "I-QT",
    10: "B-TI",
    11: "I-TI",
    12: "O"
}

label2id = {
    "B-DT": 0,
    "I-DT": 1,
    "B-LC": 2,
    "I-LC": 3,
    "B-OG": 4,
    "I-OG": 5,
    "B-PS": 6,
    "I-PS": 7,
    "B-QT": 8,
    "I-QT": 9,
    "B-TI": 10,
    "I-TI": 11,
    "O": 12

}

In [15]:
# 값 지정하여 모델 불러오기
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "klue/bert-base",
    num_labels=13,
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# 추론값 로짓 형태로 출력
with torch.no_grad():
    logits = model(**batch).logits

predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

predicted_token_class

['I-DT',
 'B-LC',
 'B-QT',
 'I-PS',
 'I-PS',
 'B-QT',
 'B-QT',
 'B-QT',
 'B-QT',
 'B-LC',
 'I-TI',
 'I-LC',
 'I-PS',
 'I-PS',
 'B-TI',
 'I-PS',
 'B-QT',
 'B-LC',
 'I-DT',
 'B-QT',
 'B-OG',
 'I-DT',
 'I-TI',
 'B-QT',
 'I-DT',
 'I-TI',
 'B-QT',
 'I-QT',
 'B-OG',
 'I-PS',
 'I-TI',
 'I-QT',
 'B-QT',
 'B-TI',
 'I-PS',
 'I-TI',
 'I-PS',
 'I-PS',
 'I-PS',
 'I-DT',
 'I-PS',
 'I-PS',
 'B-QT',
 'B-QT',
 'I-DT',
 'I-TI',
 'I-TI',
 'I-TI',
 'I-PS',
 'I-PS',
 'I-DT',
 'I-DT',
 'I-PS',
 'I-LC',
 'I-LC',
 'B-PS',
 'B-PS',
 'B-OG',
 'I-TI',
 'I-PS',
 'I-OG',
 'I-TI',
 'I-OG',
 'I-OG',
 'I-OG',
 'I-TI',
 'I-DT',
 'I-DT',
 'I-PS',
 'I-DT',
 'I-TI',
 'I-DT',
 'I-DT',
 'I-PS',
 'I-PS',
 'I-DT',
 'I-DT',
 'I-TI',
 'I-QT',
 'I-TI',
 'I-TI',
 'I-TI',
 'I-TI',
 'I-TI',
 'I-DT',
 'I-DT',
 'I-TI',
 'I-PS',
 'I-TI',
 'I-TI',
 'I-TI',
 'I-OG',
 'I-TI',
 'I-TI',
 'I-OG',
 'I-PS',
 'I-OG',
 'I-DT',
 'I-TI',
 'I-DT',
 'I-TI',
 'I-TI',
 'I-DT',
 'I-DT',
 'B-OG',
 'B-PS',
 'I-PS',
 'I-DT',
 'I-TI',
 'I-TI',
 'I-TI',
 

In [19]:
# 평가 지표
import evaluate

pred_labels = logits.argmax(dim=-1).view(-1).cpu().numpy()
true_labels = batch['labels'].view(-1).numpy()
pred_labels.shape, true_labels.shape

f1 = evaluate.load("f1")
f1.compute(predictions=pred_labels, references=true_labels, average='micro')

{'f1': 0.07008547008547009}